# Import

In [1]:
import re
import requests
import pandas        as pd

from bs4             import BeautifulSoup
from datetime        import datetime

In [2]:
list_auto = []

# API building

In [3]:
#Link do acesso
url = requests.get("https://www.autoscout24.com/lst/land-rover?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU")

#Salvando o objeto
content = url.content
url_final = []

#Convertendo para um objeto do tipo BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

list_auto = []

# Acessando todos os produtos

In [4]:
def gerar_links_base(prefixo, sufixo, num_iteracoes, final_url):
    links = []

    for i in range(1, num_iteracoes + 1):
        link = f"{prefixo}{i}{sufixo}{final_url}"
        links.append(link)

    return links

# construindo o link:
prefixo = "https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page="
sufixo = ""
num_iteracoes = 20
final_url = "&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU"
            

links_gerados = gerar_links_base(prefixo, sufixo, num_iteracoes, final_url)
print(links_gerados)

['https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=1&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=2&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=3&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=4&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_

## Acessando os produtos e construindo a tabela


In [5]:
# Teste dois
for link in links_gerados:
    #print(link)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get( link )

    #Salvando o objeto
    content = page.content
    #url_final = []
    
    #Convertendo para um objeto do tipo BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

    add = soup.find_all('article', re.compile('cldt-summary-full-item'))
     #Seller/Location
    seller_detail = soup.find('div', attrs={'class':re.compile('SellerInfo')})
    seller = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_name')}).get_text().strip()
    endereco = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_address')}).get_text().strip()
        
      
    for ads in add:
        #auto = soup.find('a', attrs={'class':re.compile('ListItem_title')}).get_text().strip()
        id_ads = ads['id']
        manufacturer_by = ads['data-make']
        model = ads['data-model']
        km = ads['data-mileage']
        year = ads['data-first-registration']
        price = ads['data-price']
        version = ads.find('span', attrs={'class': re.compile("ListItem_version")}).getText()
        description_car = ads.find('div', attrs={'class': re.compile("ListItem_wrapper")}).getText()
        
        list_auto.append([id_ads, manufacturer_by, model, version, km, year, price, seller, endereco])
        
        df = pd.DataFrame(list_auto, columns=['id_ads', 'manufacturer_by', 'model', 'version',
                                              'km', 'year', 'price', 'seller',
                                              'endereco']).reset_index(drop=True)
        df['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        

        

## Armazenando os dados

In [6]:
rows = df.shape[0]
print(rows)
df.to_csv('../dataset/list_auto_cleaning.csv', encoding='utf-8', sep=';')

399


# Load DataSet

In [9]:
df_scout = pd.read_csv('../dataset/list_auto_paginacao_automatica.csv', encoding='utf-8', sep=';')
df_scout_dirty = pd.read_csv('../dataset/list_auto_cleaning.csv', encoding='utf-8', sep=';')


In [10]:
df_scout_dirty.head()

,Unnamed: 0,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime
0,0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,defender,110 SW SE,81000,08-2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
1,1,840de0a3-82a9-454c-a725-508e211af7de,land rover,defender,Defender 90 td5 -MOTORE NUOVO-MECCANICA PERFETTA,147000,11-1999,24900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
2,2,6611c967-ab67-4db5-b28f-37b77fcd332f,land rover,defender,110 SW E,79000,08-2013,39900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
3,3,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,defender,110 SW E,78000,08-2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
4,4,08c56b66-b680-4f5c-b63a-33ce190b65eb,land rover,defender,110 SW SE,52000,08-2014,42600,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
